In [1]:
print("Importing packages...")
from openicl import DatasetReader, PromptTemplate, TopkRetriever,GenInferencer
import openai
import os
from dotenv import load_dotenv
from datasets import load_dataset, Dataset
import random


Importing packages...


/home/gadmiraal/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

In [3]:
print("Loading dataset...")
dataset = load_dataset("gpt3mix/sst2")


Loading dataset...


Found cached dataset sst2 (/home/gadmiraal/.cache/huggingface/datasets/gpt3mix___sst2/default/0.0.0/90167692658fa4abca2ffa3ede1a43a71e2bf671078c5c275c64c4231d5a62fa)
100%|██████████| 3/3 [00:00<00:00, 269.12it/s]


In [4]:
dictionary_list = dataset['train'].select_columns(['text']).to_list()
sentence_list = [entry['text'] for entry in dictionary_list]
processed_list = []

for sentence in sentence_list:
    split = sentence.split()
    mask_id = random.randint(0, len(split)-1)
    split[mask_id] = '$MASK$'
    processed = ' '.join(split)
    processed_list.append({'text': processed, 'label': sentence})

new_ds = Dataset.from_list(processed_list)

In [5]:
data = DatasetReader(new_ds, input_columns=['text'], output_column='label', ds_size=100)

In [6]:
template = PromptTemplate(
    '</E> Replace in the following sentence the $MASK$ token for a single word: </Masked>.\n Solution: </Solution>',
    {'text': '</Masked>', 'label': '</Solution>'},
    ice_token='</E>'
    )

retriever = TopkRetriever(data, ice_num=3)


[2023-06-08 15:59:39,832] [openicl.icl_retriever.icl_topk_retriever] [INFO] Creating index for index set...
100%|██████████| 100/100 [00:04<00:00, 24.58it/s]


In [7]:
infr = GenInferencer(api_name='gpt3', engine='text-davinci-003', sleep_time=3)

print("Running and calculating score...")
print(infr.inference(retriever, ice_template=template))

[2023-06-08 15:59:46,980] [openicl.icl_retriever.icl_topk_retriever] [INFO] Embedding test set...


Running and calculating score...


100%|██████████| 100/100 [00:03<00:00, 29.48it/s]
[2023-06-08 15:59:50,385] [openicl.icl_retriever.icl_topk_retriever] [INFO] Retrieving data for test set...
100%|██████████| 100/100 [00:00<00:00, 6415.07it/s]
[2023-06-08 15:59:50,486] [openicl.icl_inferencer.icl_gen_inferencer] [INFO] Starting inference process...
100%|██████████| 100/100 [09:06<00:00,  5.47s/it]

[' Adrift , Bentley and Hudson stare and sniffle , respectively , as Ledger attempts , in vain , to prove that movie-star intensity can overcome bad hair design .', " Though this rude and crude film does deliver a few gut-busting laughs , its digs at modern society are all things we 've seen before .", ' An old-fashioned scary movie , one that relies on lingering terror punctuated by sudden shocks and not constant bloodshed punctuated by flying guts .', " ... has about 3\\/4th the fun of its spry 2001 predecessor -- but it 's a rushed , slapdash , sequel-for-the-sake - of-a-sequel with less than half the plot and ingenuity .", " It's a stale , overused cocktail using the same olives since 1962 as garnish .", ' You can practically hear George Orwell turning over .', " Seeing as the film lacks momentum and its position remains mostly undeterminable , the director 's experiment is a successful one .", ' Stupid , infantile , redundant , sloppy , over-the-top , and amateurish .', ' Successf